In [ ]:
from __future__ import division
%matplotlib inline
import matplotlib.pyplot as plt
from postBornCrossSpectra import PostBorn_Bispec
import numpy as np
import Cosmology as C
import pickle
from scipy.integrate import simps
from scipy.interpolate import RectBivariateSpline
from classy import Class

import copy
import kernels
plt.style.use(['seaborn-colorblind','paper'])
plt.style.use('classic')

In [ ]:
path = '/home/nessa/Documents/Projects/LensingBispectrum/CMB-nonlinear/CrossPostBorn/data/class_postlimberdensity/diagnostic/'

In [ ]:
filename = 'diagnostic_diracz1nz40_cl.dat'
corrs    = np.loadtxt(path+filename)
L        = corrs[:,0]
corrs_delta = corrs[:,8:828]



In [ ]:
filename = 'diagnostic_tophatz1_cl.dat'
corrs    = np.loadtxt(path+filename)
L        = corrs[:,0]
corrs_tophat = corrs[:,8]



In [ ]:
zs = [ 0.9       , 0.90512821, 0.91025641, 0.91538462, 0.92051282, 0.92564103, 0.93076923, 0.93589744, 0.94102564, 0.94615385,0.95128205, 0.95641026, 0.96153846, 0.96666667, 0.97179487,       0.97692308, 0.98205128, 0.98717949, 0.99230769, 0.9974359 ,       1.0025641 , 1.00769231, 1.01282051, 1.01794872, 1.02307692,      1.02820513, 1.03333333, 1.03846154, 1.04358974, 1.04871795,       1.05384615, 1.05897436, 1.06410256, 1.06923077, 1.07435897,       1.07948718, 1.08461538, 1.08974359, 1.09487179, 1.1]
zs1, zs2 = np.meshgrid(zs,zs)
print(len(zs))

In [ ]:
def set_tophat(xmin,xmax):
    
    norm = 1./(xmax-xmin)
    
    def tophat(x):
        result = np.zeros(x.shape)
        
        result[(x>xmin)*(x<xmax)]=norm
        
        return result
    return tophat

In [ ]:
indices = np.zeros((corrs_delta.shape[1],2))
count = 0
for ii in np.arange(40):
    for nn in np.arange(ii,40):
        indices[count]=[ii,nn]
        count+=1
print(count)
print(corrs_delta.shape)
print(len(indices))

In [ ]:
def get_matrix_at_L(Lind):
    interp_matrix = np.zeros((len(zs),len(zs)))
    
    for ii in range(0,len(zs)):
        
        for jj in range(ii,len(zs)): #track correlation only for delta z = 0.5

                index = np.all(np.equal(indices,[ii,jj]),axis=1)
                #print(ii,jj,np.where(index))
                val = corrs_delta[Lind,index]
                  
                interp_matrix[ii,jj] = val

                interp_matrix[jj,ii] = interp_matrix[ii,jj] #hopefully creates only a pointer :)
    return interp_matrix


interp_matrix100  = get_matrix_at_L(100-2)
interp_matrix1000 = get_matrix_at_L(1000-2)
interp_matrix10   = get_matrix_at_L(10-2)
interp_matrix2   = get_matrix_at_L(0)


In [ ]:
data  = C.CosmoData(C.Planck2015[1],np.linspace(0.,5.,400))
chi   = data.chi(zs)

In [ ]:
max_int = len(zs)

plt.figure()
plt.imshow(np.log(interp_matrix2)[0:max_int,0:max_int],interpolation='none')#,vmin=-20,vmax=1)
plt.imshow(np.log(-interp_matrix2)[0:max_int,0:max_int],interpolation='none')
plt.colorbar()
plt.show()


plt.figure()
plt.imshow(np.log(interp_matrix10)[0:max_int,0:max_int],vmin=-8,vmax=-1.0,interpolation='none')#,vmin=-20,vmax=1)
plt.imshow(np.log(-interp_matrix10)[0:max_int,0:max_int],vmin=-8,vmax=-1.0,interpolation='none')
plt.colorbar()
plt.show()

plt.figure()
plt.imshow(np.log(interp_matrix100)[0:max_int,0:max_int],vmin=-8,vmax=-1.0,interpolation='none')
plt.imshow(np.log(-interp_matrix100)[0:max_int,0:max_int],vmin=-8,vmax=-1.,interpolation='none')
plt.colorbar()
plt.show()


plt.figure()
plt.imshow(np.log(interp_matrix1000)[0:max_int,0:max_int],vmin=-8,vmax=-1.8,interpolation='none')#,vmin=-20,vmax=1)
plt.imshow(np.log(-interp_matrix1000)[0:max_int,0:max_int],vmin=-8,vmax=-1.8,interpolation='none')
plt.colorbar()
plt.show()

In [ ]:
plt.figure()
plt.imshow(np.log(interp_matrix10)[-max_int:-1,-max_int:-1],vmin=-8,vmax=-3,interpolation='none')#,vmin=-20,vmax=1)
plt.imshow(np.log(-interp_matrix10)[-max_int:-1,-max_int:-1],vmin=-8,vmax=-3,interpolation='none')
plt.colorbar()
plt.show()

plt.figure()
plt.imshow(np.log(interp_matrix100)[-max_int:-1,-max_int:-1],vmin=-8,vmax=-3,interpolation='none')
plt.imshow(np.log(-interp_matrix100)[-max_int:-1,-max_int:-1],vmin=-8,vmax=-3,interpolation='none')
plt.colorbar()
plt.show()


plt.figure()
plt.imshow(np.log(interp_matrix1000)[-max_int:-1,-max_int:-1],vmin=-8,vmax=-3,interpolation='none')
plt.imshow(np.log(-interp_matrix1000)[-max_int:-1,-max_int:-1],vmin=-8,vmax=-3,interpolation='none')
plt.colorbar()
plt.show()

In [ ]:
window = set_tophat(min(zs),max(zs))
chi1, chi2 = np.meshgrid(chi,chi)

result =[]
for L_ind in np.arange(0,2999):
    print(L_ind)
    LL = L[L_ind]
    matrix          = get_matrix_at_L(L_ind)
    weighted_matrix = window(zs1)*data.dchidz(zs1)**(-1)*window(zs2)*data.dchidz(zs2)**(-1)*matrix#Cl_psidelta(chi1,zs1,L[L_ind],matrix,cosmo=data)
    
    res = []
    for ii in range(len(zs)):
        res+=[np.trapz(weighted_matrix[ii],chi)]

    
    result+=[np.trapz(res,chi)]


In [ ]:
print(data.dchidz(zs2)**(-1)*data.dchidz(zs1)**(-1))

In [ ]:
print(result)

In [ ]:
plotpath = '/home/nessa/Documents/Projects/LensingBispectrum/CMB-nonlinear/CrossPostBorn/results/plots/vanessa/'

In [ ]:
plt.figure()
plt.loglog(L,corrs_tophat,label='CLASS')
plt.loglog(L,np.asarray(result),label='our test')
plt.xlim(min(L),1000)
plt.legend(loc='lower left')
plt.savefig(plotpath+'tophat_comparison_baseline.png')
plt.show()

In [ ]:
plt.figure()
plt.semilogx(L,np.asarray(result)/corrs_tophat-1,label='CLASS/our test -1 ')
plt.xlim(min(L),1000)
plt.legend(loc='lower right')
plt.ylim(-0.0,0.5)
plt.savefig(plotpath+'tophat_comparison_baseline_relative.png')